In [22]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], 
   [0.55, 0.87, 0.66],
   [0.57, 0.85, 0.64], 
   [0.22, 0.58, 0.33], 
   [0.77, 0.25, 0.10], 
   [0.05, 0.80, 0.55]] 
)

In [23]:
context_vectors = torch.zeros(inputs.shape)
for k in range(inputs.size(0)):
    query = inputs[k]
    attention_weights = torch.zeros(inputs.size(0))
    context_vector = torch.zeros(query.size(0))

    for i in range(inputs.size(0)):
        attention_weights[i] = torch.dot(inputs[i],query)
        
    attention_weights = torch.softmax(attention_weights,dim=0)

    for j in range(inputs.size(0)):
        context_vector += inputs[j]*attention_weights[j]
    
    context_vectors[k] = context_vector
    
print(context_vectors)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [24]:
att_weights = inputs@inputs.T
att_weights = torch.softmax(att_weights,dim=1)
c_vectors = att_weights@inputs
print(c_vectors)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


##### Attention with Trainable Weights

In [27]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

W_q = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_k = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_v = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)

print(W_q.size())
print(x_2.size())

torch.Size([3, 2])
torch.Size([3])


In [28]:
query_2 = x_2@W_q
key_2 = x_2@W_k
value_2 = x_2@W_v

print(query_2.size())

torch.Size([2])


In [30]:
key = inputs@W_k
value = inputs@W_v

print(key.size())
print(value.size())

torch.Size([6, 2])
torch.Size([6, 2])


In [35]:
attention_scores_2 = query_2@key.T
print(attention_scores_2)

tensor([1.9417, 2.9210, 2.9031, 1.5773, 1.7641, 1.8577])


In [36]:
d_k = key.size(-1)
attention_weights_2 = torch.softmax(attention_scores_2/(d_k**0.5),dim=-1)
print(attention_weights_2)

tensor([0.1321, 0.2640, 0.2607, 0.1021, 0.1165, 0.1245])


In [37]:
context_vector_2 = attention_weights_2@value
print(context_vector_2)

tensor([0.6109, 0.7207])


##### Generalizing computation of attention across all inputs

In [40]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], 
   [0.55, 0.87, 0.66],
   [0.57, 0.85, 0.64], 
   [0.22, 0.58, 0.33], 
   [0.77, 0.25, 0.10], 
   [0.05, 0.80, 0.55]] 
)

d_in = inputs.size(1)
d_out = 2

w_q = torch.nn.Parameter(torch.rand(d_in,d_out))
w_k = torch.nn.Parameter(torch.rand(d_in,d_out))
w_v = torch.nn.Parameter(torch.rand(d_in,d_out))

query = inputs@w_q
key = inputs@w_k
value = inputs@w_v

d_k = key.size(-1)
attention_scores = query@key.T
attention_weights = torch.softmax(attention_scores/(d_k**0.5),dim=-1)
context_vector = attention_weights@value
context_vector

tensor([[0.8691, 0.8326],
        [0.8792, 0.8426],
        [0.8789, 0.8422],
        [0.8561, 0.8192],
        [0.8598, 0.8229],
        [0.8610, 0.8242]], grad_fn=<MmBackward0>)

In [102]:
torch.manual_seed(789)
class SelfAttention(torch.nn.Module):
    def __init__(self,d_in,d_out,dropout,qkv_bias=False):
        super(SelfAttention,self).__init__()
        self.w_q = torch.nn.Linear(d_in,d_out,bias=qkv_bias)
        self.w_k = torch.nn.Linear(d_in,d_out,bias=qkv_bias)
        self.w_v = torch.nn.Linear(d_in,d_out,bias=qkv_bias)
        self.dropout = torch.nn.Dropout(dropout)
       
        
    def forward(self,inputs): 
        query = self.w_q(inputs)
        key = self.w_k(inputs)
        value = self.w_v(inputs)
        d_k = key.size(-1)
        attention_scores = query@key.T
        
        mask = torch.triu(torch.ones(attention_scores.size(0),attention_scores.size(0)),diagonal=1)
        masked_attention_scores = attention_scores.masked_fill(mask.bool(),-torch.inf)
        
        attention_weights = torch.softmax(masked_attention_scores/(d_k**0.5),dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        context_vector = attention_weights@value
        return context_vector

In [103]:
self_attention = SelfAttention(inputs.size(1),2,0.5)
print(self_attention(inputs))

tensor([[-0.1744,  0.0572],
        [ 0.0000,  0.0000],
        [-0.0632,  0.0576],
        [-0.1464,  0.0522],
        [-0.0855,  0.0672],
        [-0.0716,  0.0365]], grad_fn=<MmBackward0>)


In [104]:
class CausalAttention(torch.nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = torch.nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool(), -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

In [106]:
intp = torch.stack((inputs,inputs))
intp.shape
causal_attention = CausalAttention(intp.size(2),2,6,0.5)
causal_attention(intp)

tensor([[[ 0.0000,  0.0000],
         [-0.1615, -0.5010],
         [ 0.0186, -0.0748],
         [-0.0865, -0.2086],
         [-0.0534, -0.2567],
         [-0.0535, -0.1835]],

        [[-0.3596, -0.7687],
         [ 0.0183, -0.1166],
         [-0.1076, -0.3341],
         [ 0.0295, -0.1277],
         [ 0.0168, -0.0576],
         [ 0.0033, -0.1458]]], grad_fn=<UnsafeViewBackward0>)

In [107]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = torch.nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec